In [332]:
import os
from os.path import join as opj
import numpy as np
import random

import open3d as o3d
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt
import trimesh

from utils import *
from dataset import *

In [333]:
import pickle as pkl
data_root = 'data_root'
new_aff_path = opj(data_root, 'remapped_full_shape_train_data.pkl')
with open(new_aff_path, 'rb') as f:
    new_aff_dataset = pkl.load(f)
print("Loaded dataset with {} objects".format(new_aff_dataset['num_objects']))
print("Num of semantic classes: ", len(new_aff_dataset["semantic_classes"]))
print("Semantic classes: ", new_aff_dataset["semantic_classes"])
print("Num of affordances: ", len(new_aff_dataset["affordance_classes"]))
print("All affordances: ", new_aff_dataset["affordance_classes"])

Loaded dataset with 1096 objects
Num of semantic classes:  7
Semantic classes:  ['Knife', 'Bag', 'Bottle', 'Scissors', 'Mug', 'Earphone', 'Hat']
Num of affordances:  7
All affordances:  ['handover', 'cut', 'stab', 'lift', 'wrap', 'pour', 'wear']


In [334]:
## pick a random datapoint
# set_random_seed(0)
# pick a random sem class
sem_class = random.choice(new_aff_dataset['semantic_classes'])
# pick a random shape id
shape_id = random.choice(list(new_aff_dataset['data'][sem_class].keys()))
datapoint = new_aff_dataset['data'][sem_class][shape_id]

In [335]:
print("Shape id ", shape_id, "\nSemantic class: ", sem_class)
pcl = datapoint['coordinates']
# Viz pcl with o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcl)
# color black
pcd.paint_uniform_color([0, 0, 0])
# viz in jupyter nb
print("Drawing pcd with", len(pcl), "points")
draw(pcd)

Shape id  6e884701bfddd1f71e1138649f4c219 
Semantic class:  Mug
Drawing pcd with 2048 points
[Open3D INFO] Window window_229 created.


WebVisualizer(window_uid='window_229')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate


In [ ]:
# get pcls of all labels
labels_list = []
scores_list = []
for key in datapoint['labels'].keys():
    labels = datapoint['labels'][key]
    scores = labels.sum()
    print("Label: ", key, " Score: ", scores)
    labels_list.append(labels)
    scores_list.append(scores)

    labelled_pcd = o3d.geometry.PointCloud()
    labelled_pcd.points = o3d.utility.Vector3dVector(pcl)
    labelled_pcd.paint_uniform_color([0, 0, 0])
    np.asarray(labelled_pcd.colors)[labels.squeeze() > 0] = np.random.uniform(0, 1, 3)
    print("Label 0: ", key)
    draw([labelled_pcd])

Label:  handover  Score:  355.65207
Label 0:  handover
[Open3D INFO] Window window_230 created.


WebVisualizer(window_uid='window_230')

Label:  wrap  Score:  89.07495
Label 0:  wrap
[Open3D INFO] Window window_231 created.


WebVisualizer(window_uid='window_231')

Label:  pour  Score:  89.07495
Label 0:  pour
[Open3D INFO] Window window_232 created.


WebVisualizer(window_uid='window_232')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.6831523545538223
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.6831523545538223
[Open3D INFO] Sending init frames to window_229.
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.7946895901193369
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.7946895901193369
[Open3D INFO] Sending init

[2179:762][12701] (stun_port.cc:96): Binding request timed out from 130.83.164.x:54200 (enp5s0)
[2180:586][12701] (stun_port.cc:96): Binding request timed out from 130.83.164.x:48852 (enp5s0)
[2180:593][12701] (stun_port.cc:96): Binding request timed out from 130.83.164.x:48981 (enp5s0)
[2180:600][12701] (stun_port.cc:96): Binding request timed out from 130.83.164.x:35553 (enp5s0)
[2180:613][12701] (stun_port.cc:96): Binding request timed out from 130.83.164.x:50851 (enp5s0)


In [337]:
# Load and visualize the shapenet object
from utils.partnet_utils import cat_descs
# make keys values and values keys
SHAPENET_PATH = '/home/sjauhri/IAS_WS/ShapeNetCore.v2'
obj_path = opj(SHAPENET_PATH, cat_descs[sem_class.lower()],
                shape_id, 'models', 'model_normalized.obj')
# Load with trimesh
# obj_mesh = trimesh.load(obj_path)
# obj_mesh.show()
# load with open3d
obj_mesh = o3d.io.read_triangle_mesh(obj_path)

## Re-center and re-scale the corresponding point cloud to match the original shape

orig_points = obj_mesh.sample_points_uniformly(number_of_points=8000).points
# get the center of the mesh
orig_center = np.mean(orig_points, axis=0)
# get the maximum extent of the mesh
centered_points = orig_points - orig_center # center the points
orig_max_dist = np.max(np.sqrt(np.sum(centered_points**2, axis=1)))

# correct the pcl
corrected_pcl = pcl.copy()
# # flip along the z axis??
corrected_pcl[:,2] = -corrected_pcl[:,2]
# # flip along the x axis??
corrected_pcl[:,0] = -corrected_pcl[:,0]
# scale the point cloud
corrected_pcl = corrected_pcl * orig_max_dist
# recenter the point cloud
corrected_pcl += orig_center

# Viz with o3d/trimesh
corrected_pcd = o3d.geometry.PointCloud()
corrected_pcd.points = o3d.utility.Vector3dVector(corrected_pcl)
# color red
corrected_pcd.paint_uniform_color([1, 0, 0])
# draw([obj_mesh,corrected_pcl_pcd])
axes_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6)
draw([obj_mesh, corrected_pcd, axes_mesh])

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Window window_233 created.


WebVisualizer(window_uid='window_233')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.06872436164190932
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.06872436164190932
[Open3D INFO] Sending init frames to window_233.
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
